## Problem 5

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf

# shortest path code is in this module
from P5_sssp import *

# setup spark
conf = SparkConf().setAppName('WikiGraph')
sc = SparkContext(conf=conf, pyFiles=['P5_sssp.py'])
sc.setLogLevel('ERROR')

In [2]:
# prepare test graph
# graph should have no jumps in node IDs! (maybe remap before!)
testgraph = [(1, [2, 3]), (2, [1, 3]), (3, [1,2,4]), (4, [3]), (5, [6, 7]), (6, [5]), (7, [5])]
rdd = sc.parallelize(testgraph)

In [3]:
# transform to structure (init)
rddA = rdd.map(lambda x: (x[0], (x[1], x[0])))
maxNodeID = rddA.map(lambda x: x[0]).max()
minNodeID = rddA.map(lambda x: x[0]).min()

In [4]:
rddA.map(lambda x: x[0]).min()

1

In [5]:
rddA.collect()

[(1, ([2, 3], 1)),
 (2, ([1, 3], 2)),
 (3, ([1, 2, 4], 3)),
 (4, ([3], 4)),
 (5, ([6, 7], 5)),
 (6, ([5], 6)),
 (7, ([5], 7))]

In [6]:
# helper function to check whether the partitioning in connected components changed
def checkForPartitionChange(newSizes, oldSizes):
    partitionChanged = False

    # quick check if length of the dict is different to previous step
    # the partitions changed!
    if len(oldSizes.items()) != len(newSizes.items()):
        partitionChanged = True
    else:
        # the size did not change, but what about some internal shifting?
        # ==> check for each connected component if size changed!
        for ID in range(minNodeID, maxNodeID+1):
            # check if for old / new the current ID exists and is equal
            # ==> if not, a change happened!
            try:
                sizeOld = oldSizes[ID]
                sizeNew = newSizes[ID]

                if sizeOld != sizeNew:
                    partitionChanged = True
                    break
            except KeyError:
                partitionChanged = True
                break
                
    return partitionChanged

In [7]:

rdd = rddA


# this is the algorithm (Pegasus after ...)
done = False

while not done:
    # compute sizes of connected components
    rddComponents = rdd.map(lambda x: x[1][1])
    oldSizes = rddComponents.countByValue()


    # mapper / expander
    rdd = rdd.flatMap(lambda x: [x] + [(y, ([], x[1][1])) for y in x[1][0]])

    # reducer (as there is only !one! element with the whole adj. list, we can speed it up here!)
    rdd = rdd.reduceByKey(lambda a,b: (a[0] + b[0], min(a[1], b[1])))

    # determine if number of partitions changed! (0 is a dummy value to construct an pair rdd)
    rddComponents = rdd.map(lambda x: x[1][1])
    newSizes = rddComponents.countByValue()

    # if partition changed, one more round!
    done = not checkForPartitionChange(newSizes, oldSizes)

    oldSizes = newSizes

# reconstruct connected components
componentList = rdd.map(lambda x: (x[1][1], x[0])).sortByKey().groupByKey().map(lambda x: sorted(list(x[1]))).collect()


In [22]:
# perform PEGASUS algorithm after ...
def connectedComponents(rddIn):
    
    # transform to structure (init)
    rdd = rddIn.map(lambda x: (x[0], (x[1], x[0])))
    maxNodeID = rdd.map(lambda x: x[0]).max()
    minNodeID = rdd.map(lambda x: x[0]).min()
    
    # helper function to check whether the partitioning in connected components changed
    def checkForPartitionChange(newSizes, oldSizes):
        partitionChanged = False

        # quick check if length of the dict is different to previous step
        # the partitions changed!
        if len(oldSizes.items()) != len(newSizes.items()):
            partitionChanged = True
        else:
            # the size did not change, but what about some internal shifting?
            # ==> check for each connected component if size changed!
            for ID in range(minNodeID, maxNodeID+1):
                # check if for old / new the current ID exists and is equal
                # ==> if not, a change happened!
                try:
                    sizeOld = oldSizes[ID]
                    sizeNew = newSizes[ID]

                    if sizeOld != sizeNew:
                        partitionChanged = True
                        break
                except KeyError:
                    partitionChanged = True
                    break

        return partitionChanged
    
    
    # this is the algorithm (Pegasus after http://arxiv.org/pdf/1203.5387.pdf)
    done = False
    
    counter = 0;
    while not done:
        # compute sizes of connected components
        rddComponents = rdd.map(lambda x: x[1][1])
        oldSizes = rddComponents.countByValue()


        # mapper / expander
        rdd = rdd.flatMap(lambda x: [x] + [(y, ([], x[1][1])) for y in x[1][0]])

        # reducer (as there is only !one! element with the whole adj. list, we can speed it up here!)
        rdd = rdd.reduceByKey(lambda a,b: (a[0] + b[0], min(a[1], b[1])))
        
        # determine if number of partitions changed! (0 is a dummy value to construct an pair rdd)
        rddComponents = rdd.map(lambda x: x[1][1])
        newSizes = rddComponents.countByValue()

        # if partition changed, one more round!
        done = not checkForPartitionChange(newSizes, oldSizes)

        oldSizes = newSizes
        
        counter += 1

    # reconstruct connected components
    componentList = rdd.map(lambda x: (x[1][1], x[0])).sortByKey().groupByKey().map(lambda x: sorted(list(x[1]))).collect()

    return counter, componentList

In [17]:
# test run on rdd!
rddTest = sc.parallelize(testgraph)

cList = connectedComponents(rddTest)

In [18]:
cList

(3, [[1, 2, 3, 4], [5, 6, 7]])

In [19]:
# Test run for Marvel Graph

# this function prepares the rdd
def prepare_rdd(filename):
    rdd = sc.textFile(filename)

    # map string to tuples
    rdd = rdd.map(lambda x: x.split(','))
    rdd = rdd.map(lambda x: (int(x[0]), int(x[1])))
    
    # now group s.t. we have for each vertex an adjacency list of nodes
    rdd = rdd.groupByKey().map(lambda x: (x[0], list(x[1])))
    
    return rdd

In [20]:
# test code
filename = 'edge_list.csv'#'edge_list_simple.csv' # 'edge_list.csv'

rdd = prepare_rdd(filename)

In [23]:
# now find connected components!
num_iterations, cList = connectedComponents(rdd)

In [24]:
len(cList)

4

In [25]:
num_iterations

5

## conversion function for directedGraph to undirected one

In [26]:
# make each connection symmetric

# prepare test graph
# graph should have no jumps in node IDs! (maybe remap before!)
testgraph = [(1, [2, 3]), (2, [3]), (3, [2]), (4, [3]), (5, []), (6, [5]), (7, [5])]
rdd = sc.parallelize(testgraph)


In [27]:
rdd = rdd.flatMap(lambda x: [x] + [(y, [x[0]]) for y in x[1]])

In [28]:
rdd = rdd.reduceByKey(lambda a,b: list(set(a) | set(b)))

In [29]:
rdd.collect()

[(4, [3]),
 (1, [2, 3]),
 (5, [6, 7]),
 (2, [1, 3]),
 (6, [5]),
 (3, [1, 2, 4]),
 (7, [5])]

In [30]:
# all in one function
def directedGraph2symmetric(rdd):
    return rdd.flatMap(lambda x: [x] + [(y, [x[0]]) for y in x[1]]).reduceByKey(lambda a,b: a+b).map(lambda x: (x[0], list(set(x[1]))))

In [31]:
# test code for the function
testgraph = [(1, [2, 3]), (2, [3]), (3, [2]), (4, [3]), (5, []), (6, [5]), (7, [5])]
rdd = sc.parallelize(testgraph)
directedGraph2symmetric(rdd).collect()

[(4, [3]),
 (1, [2, 3]),
 (5, [6, 7]),
 (2, [1, 3]),
 (6, [5]),
 (3, [1, 2, 4]),
 (7, [5])]

In [32]:
# test code for the function
testgraph = [(1, [2, 3]), (2, [3]), (3, [2, 4]), (4, [3]), (5, []), (6, [5]), (7, [5])]
rdd = sc.parallelize(testgraph)
rdd.collect()

[(1, [2, 3]), (2, [3]), (3, [2, 4]), (4, [3]), (5, []), (6, [5]), (7, [5])]

In [33]:
rddSingleNodes = rdd.map(lambda x: (x[0], []))
rdd = rdd.flatMap(lambda x: [(x[0], y) for y in x[1]])

In [34]:
rdd = rdd.map(lambda x: ((min(x[0], x[1]), max(x[0], x[1])), 1))

In [35]:
rdd.collect()

[((1, 2), 1),
 ((1, 3), 1),
 ((2, 3), 1),
 ((2, 3), 1),
 ((3, 4), 1),
 ((3, 4), 1),
 ((5, 6), 1),
 ((5, 7), 1)]

In [36]:
rdd = rdd.reduceByKey(lambda a,b: a+b).filter(lambda x: x[1] > 1).map(lambda x: x[0])

In [37]:
rdd = rdd.flatMap(lambda x: [(x[0], [x[1]]), (x[1], [x[0]])])
# if desired, join with single nodes
rdd = rdd.union(rddSingleNodes)

rdd = rdd.reduceByKey(lambda a, b: a+b)

In [38]:
rdd.collect()

[(1, []), (2, [3]), (3, [2, 4]), (4, [3]), (5, []), (6, []), (7, [])]

WICHTIG!!! Bei dem zweiten, unbedingt die Zusammenhangskomponenten mit Groesse 1 zuerst rausfiltern! Damit ist der Algorithmus schneller!!!

In [39]:
# all in one function\
def directedGraph2linked(rdd, includeSingleComponents=True):
    rddSingleNodes = None
    if includeSingleComponents:
        rddSingleNodes = rdd.map(lambda x: (x[0], []))
        
    rdd = rdd.flatMap(lambda x: [(x[0], y) for y in x[1]])
    rdd = rdd.map(lambda x: ((min(x[0], x[1]), max(x[0], x[1])), 1))
    rdd = rdd.reduceByKey(lambda a,b: a+b).filter(lambda x: x[1] > 1).map(lambda x: x[0])
    rdd = rdd.flatMap(lambda x: [(x[0], [x[1]]), (x[1], [x[0]])])
    
    if includeSingleComponents:
        # if desired, join with single nodes
        rdd = rdd.union(rddSingleNodes)

    rdd = rdd.reduceByKey(lambda a, b: a+b)
    
    return rdd

In [40]:
# test code for the function
testgraph = [(1, [2, 3]), (2, [3]), (3, [2, 4]), (4, [3]), (5, []), (6, [5]), (7, [5])]
rdd = sc.parallelize(testgraph)
rdd.collect()

directedGraph2linked(rdd, False).collect()

[(4, [3]), (2, [3]), (3, [2, 4])]

# Wikipedia Graph

In [42]:
# paths to the big data files
datapath = '../../../../../courses/CS205_Computing_Foundations/data/'
titlespath = datapath + 'titles-sorted.txt'
linkspath = datapath + 'links-simple-sorted.txt'

In [45]:
# function to prepare two rdds, one holding the graph, the other for later use as a dictionary
def prepareWikiGraph(titlefile, linksfile):
	rddTitles = sc.textFile(titlefile)
	rddGraph = sc.textFile(linksfile)

	# title file has structure v: v0, ...., v_d
	# simple mapping will give the rdd structure
	# (v, [v_1, ..., v_d]) as needed by the sssp algorithm
	rddGraph = rddGraph.map(lambda x: x.replace(':', '').split(' ')) \
					   .map(lambda x: (int(x[0]), [int(y) for y in x[1:]]))

	# note that for the wikigraph everything is 1-indexed
	# dictionary has structure ('a wiki title', 23)
	rddTitles = rddTitles.zipWithIndex().map(lambda x: (x[0], x[1] + 1)).cache()
	return rddGraph, rddTitles

In [43]:
rddGraph, rddTitles = prepareWikiGraph(titlespath, linkspath) 

# cache graph for better performance
rddGraph = rddGraph.cache()

# first compute connected components for the linked graph (it is smaller)
rddLinked = directedGraph2linked(rddGraph, False)

# now find connected components!
num_iterations, cList = connectedComponents(rddGraph)


In [44]:
len(cList) # add to this number 1 -components

4